In [ ]:
# @title
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import geopandas as gpd
import folium
GeoPoints = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/all_v2.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Polygons = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/geo.json')

region_url = "/content/drive/MyDrive/Colab Notebooks/geo.json" #folium.Choropleth принимает географические данные только в json формате

In [ ]:
GeoPoints.columns

Index(['price', 'date', 'time', 'geo_lat', 'geo_lon', 'region',
       'building_type', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'object_type'],
      dtype='object')

In [ ]:
GeoPoints = gpd.GeoDataFrame(geometry=gpd.points_from_xy(GeoPoints.geo_lon, GeoPoints.geo_lat, crs="EPSG:4326"), data=GeoPoints)
GeoPoints = GeoPoints.drop(['object_type', 'region'], axis=1)

In [ ]:
GeoPoints = gpd.sjoin(GeoPoints, Polygons, predicate = 'within')

In [ ]:
data_aggr = GeoPoints[['price', 'area', 'NAME_1']]
del GeoPoints

In [ ]:
data_aggr.sort_values(by='price')

,price,area,NAME_1
4427261,-2144967296,33.60,Novosibirsk
4426359,-2144967296,33.60,Novosibirsk
3829586,-2114967296,43.40,Novosibirsk
3816852,-2114967296,43.40,Novosibirsk
1895619,-2114150296,87.39,Novosibirsk
...,...,...,...
1429415,2083290000,63.00,Novosibirsk
1396657,2089477704,95.86,Krasnodar
1053295,2089477704,95.86,Krasnodar
1142170,2147483647,50.00,Moskva


Удаляем минусовые и нулевые значения


In [ ]:
data_aggr = data_aggr.query('price !=0 and price > 0')

In [ ]:
data_aggr['price_m2'] = round(data_aggr.price / data_aggr.area)

<ipython-input-88-7d155cb24372>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aggr['price_m2'] = round(data_aggr.price / data_aggr.area)


Вычисляем медиану для получения более точного значение стоимости квадратного метра в регионе


In [ ]:
data_aggr = data_aggr.groupby(['NAME_1'] , as_index=False).aggregate({'price_m2':'median'}).round(1)

In [ ]:
data_aggr

,NAME_1,price_m2
0,Adygey,46000.0
1,Altay,44405.0
2,Amur,85595.0
3,Arkhangel'sk,74583.0
4,Astrakhan',43750.0
...,...,...
79,Voronezh,49464.0
80,Yamal-Nenets,68571.0
81,Yaroslavl',47143.0
82,Yevrey,42453.0


In [ ]:
m = folium.Map(
    location = [30.376141, 59.8058084],    # широта и долгота России
    zoom_start = 3,tiles="OpenStreetMap")

In [ ]:
Choropleth = folium.Choropleth(
		data = data_aggr,
		geo_data = region_url,
		columns=['NAME_1', 'price_m2'],
		key_on='feature.properties.NAME_1',
		locations = data_aggr.NAME_1,
		fill_opacity=0.7,
		line_opacity=1,
		fill_color = 'YlGn',
		legend_name="Unemployment Rate (%)",
		name = 'Недвижимость').add_to(m),
folium.LayerControl().add_to(m)

In [ ]:
Polygons["centroid"] = Polygons.centroid


<ipython-input-93-fa3147cf260f>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  Polygons["centroid"] = Polygons.centroid


Наносим в географический центр каждого региона значок с легендой:

In [ ]:
for _, r in Polygons.iterrows():
  if r['NAME_1'] != 'Chukot':
    lat = r["centroid"].y
    lon =r["centroid"].x
    folium.Marker(
        location=[lat, lon],
        popup = "Квадратный метр: {} <br>".format(data_aggr['price_m2'][data_aggr['NAME_1'] == r['NAME_1']].values[0])#Получаем данные из data_aggr для создание легенды на карте
     ).add_to(m)

In [ ]:
m

Output hidden; open in https://colab.research.google.com to view.